In [2]:
import os
import numpy as np
from keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# WITH PREPROCESSING

In [5]:
ds_train2 = image_dataset_from_directory('../data/french_bird_db_25_50', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="training",
seed=42, batch_size=64)

ds_test2 = image_dataset_from_directory('../data/french_bird_db_25_50', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="validation",
seed=42, batch_size=64)

Found 1250 files belonging to 25 classes.
Using 1000 files for training.
Found 1250 files belonging to 25 classes.
Using 250 files for validation.


In [6]:
class_names = ds_train2.class_names
num_classes = len(ds_train2.class_names)

In [9]:
# preprocessing
preprocess_input = tf.keras.applications.resnet50.preprocess_input

ds_train2_preprocessed = ds_train2.map(lambda x, y: (preprocess_input(x), y))
ds_test2_preprocessed = ds_test2.map(lambda x, y: (preprocess_input(x), y))


In [10]:
tf.config.list_physical_devices()

# Charger le modèle ResNet50 pré-entraîné avec les poids ImageNet
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Ajouter une couche de pooling global pour réduire la dimensionalité
x = base_model.output
x_2 = GlobalAveragePooling2D()(x)

x_3 = Dense(num_classes, activation='relu')(x_2)

# Ajouter une couche dense pour effectuer la classification
output = Dense(num_classes, activation='softmax')(x_3)

# Créer le modèle final en combinant le modèle de base et les couches supplémentaires
model2 = Model(inputs=base_model.input, outputs=output)

# Geler les couches du modèle de base pour éviter de les entraîner
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
# model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.compile(optimizer=Adam(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

callbacks = [EarlyStopping(monitor='val_loss', patience=10)]


# Entraîner le modèle sur votre dataset avec le callback personnalisé

history = model2.fit(ds_train2_preprocessed, epochs=10, callbacks=callbacks, validation_data=ds_test2_preprocessed)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - loss: 3.1828 - sparse_categorical_accuracy: 0.0714 - val_loss: 2.8948 - val_sparse_categorical_accuracy: 0.1720
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 2.6771 - sparse_categorical_accuracy: 0.2618 - val_loss: 2.4217 - val_sparse_categorical_accuracy: 0.3480
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 2.2065 - sparse_categorical_accuracy: 0.4699 - val_loss: 1.9415 - val_sparse_categorical_accuracy: 0.5320
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - loss: 1.7003 - sparse_categorical_accuracy: 0.5928 - val_loss: 1.5792 - val_sparse_categorical_accuracy: 0.6560
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - loss: 1.2798 - sparse_categorical_accuracy: 0.7471 - val_loss: 1.2663 - val_sparse_categorical_accuracy: 0.7360
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 0.9369 - sparse_categorical_accuracy: 0.8365 - val_loss: 1.0589 - val_sparse_categorical_accuracy: 0.7800
Epoch 7/10
16/16

In [15]:
for image_batch, labels_batch in ds_train2_preprocessed:
  batch_pred = model2.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  print(class_pred)
  print(labels_batch)
  break

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 722ms/step
[ 2  8 14  8 21 13  8  6 15 16 21  7 23 18 19 11 12 11  7 16 24 20 10 22
 16 22  3 20 23  9 13  9  8 23  7 19  2 23 10 18 11 10  2 13 21  1  4 21
 23 24  3 17  4  9 22  0 19 16  8 11  4  4  5 22]
tf.Tensor(
[ 2  8 14  8 21 13  8  6 15 16 21  7 23 18 19 11 12 11  7 16 24 13 10 22
 16 22  3 20 23  9 13  9  8 23  8 19  2  7  5 18 11 10  2 13 21  1  4 21
 23 24  3 17  4  9 22  0 19 16  8 11  4  4  5 22], shape=(64,), dtype=int32)


In [24]:
for i, (image_batch, labels_batch) in enumerate(ds_test2_preprocessed):
  batch_pred = model2.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  
  print('batch No {} with result: '.format(i), sum((class_pred == labels_batch).numpy()) / len(class_pred) * 100, '%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
batch No 0 with result:  79.6875 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step
batch No 1 with result:  78.125 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step
batch No 2 with result:  82.8125 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step  
batch No 3 with result:  87.93103448275862 %


2024-07-03 15:49:56.212735: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
